**Conocimiento estadistico:**

3. **Estadistica:** Supongamos que trabajas en una planta de fabricación y operas una máquina industrial clave en la línea de producción. Esta máquina tiene un historial de tiempos de fallo que se han registrado a lo largo de los años. Tienes un conjunto de datos que contiene los tiempos de fallo de esta máquina en horas, y deseas estimar la probabilidad de que la máquina falle antes de cierto tiempo para planificar el mantenimiento preventivo.

Calcular la probabilidad de que falle en 2000 horas

El archivo para el desarrollo del ejercicio se encuentra con el nombre de `data_supervivencia.csv`.

   **Información del dataset**
    
    - Ubic.técn.
    - Descripción del Objeto Tecnico.
    - Texto breve de material.
    - Texto breve orden.
    - Fecha.
    - Horas de Vida.
    

**Ayuda:**
Hacer uso de las tecnicas de analisis de supervivencia y use censura de datos para los tres valores mas pequeños.

# **Solución**
Una de las soluciones que se puede implementar para solventar problemas de supervivencia es el conocido algoritmo de `Kaplan Meier`.

Este es ampliamente conocido en muchos campos como la biología, la medicina, la industria, etc. El algoritmo se basa en una fórmula recursiva dada por:

$$S_t = S_{t-1} ⋅(1-\frac{d_i}{N_i})$$

donde,

 $S_t$ es la estimación de la función de supervivencia acumulativa en el tiempo $t$

 $S_{t-1}$ es la estimación de la función de supervivencia acumulativa en el tiempo $t-1$

$d$ es el número de eventos (fallas) ocurridos en el tiempo $t$

$N_{i}$ es el número de individuos en riesgo en el tiempo $t$

Pero podemos implementar este algoritmo a través de la librería `lifelines`

In [1]:
!pip install lifelines

In [2]:
!pip install scikit-survival

In [3]:
import pandas as pd
import numpy as np
from lifelines import KaplanMeierFitter

In [4]:
# Importemos el dataset
df = pd.read_csv('data_supervivencia.csv')

In [5]:
# Veamos la información contenida
df.head(2)

,Ubic.técn.,Descripción del Objeto Tecnico,Texto breve de material,Texto breve orden,Fecha,Horas de Vida
0,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV LLENADORA BOT L2,2/24/2018,-
1,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV EN LLENADORA L2,5/10/2018,"1,800.00"


In [6]:
# Veamos si hay valores nulos
df.isna().sum()

Ubic.técn.                        0
Descripción del Objeto Tecnico    0
Texto breve de material           0
Texto breve orden                 0
Fecha                             0
 Horas de Vida                    0
dtype: int64

In [7]:
# Descripcion de los datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Ubic.técn.                      14 non-null     object
 1   Descripción del Objeto Tecnico  14 non-null     object
 2   Texto breve de material         14 non-null     object
 3   Texto breve orden               14 non-null     object
 4   Fecha                           14 non-null     object
 5    Horas de Vida                  14 non-null     object
dtypes: object(6)
memory usage: 800.0+ bytes


In [8]:
# Estandazacion de la columna a utilizar
df.rename(columns = {' Horas de Vida ':'Horas de vida'}, inplace = True)

In [9]:
df.head(2)

,Ubic.técn.,Descripción del Objeto Tecnico,Texto breve de material,Texto breve orden,Fecha,Horas de vida
0,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV LLENADORA BOT L2,2/24/2018,-
1,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV EN LLENADORA L2,5/10/2018,"1,800.00"


In [10]:
# Estandarizacion de los valores de la columna
df['Horas de vida'].min()

'  -   '

In [11]:
df['Horas de vida'] = df['Horas de vida'].str.replace(r'\s+', '')

<ipython-input-11-ec355d886ac5>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Horas de vida'] = df['Horas de vida'].str.replace(r'\s+', '')


In [12]:
df['Horas de vida'].min()

'-'

In [13]:
# Reemplaza - por 0 en str
df['Horas de vida'].replace(r'-','0', inplace=True)

In [14]:
# Convierte la columna a valores numéricos
df['Horas de vida']=pd.to_numeric(df['Horas de vida'].str.replace(',', ''))

In [15]:
# Vemos de nuevo la informacion y ya la columna es flotante
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Ubic.técn.                      14 non-null     object 
 1   Descripción del Objeto Tecnico  14 non-null     object 
 2   Texto breve de material         14 non-null     object 
 3   Texto breve orden               14 non-null     object 
 4   Fecha                           14 non-null     object 
 5   Horas de vida                   14 non-null     float64
dtypes: float64(1), object(5)
memory usage: 800.0+ bytes


In [16]:
df

,Ubic.técn.,Descripción del Objeto Tecnico,Texto breve de material,Texto breve orden,Fecha,Horas de vida
0,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV LLENADORA BOT L2,2/24/2018,0.0
1,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV EN LLENADORA L2,5/10/2018,1800.0
2,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIO LAMPARA UV L-2,8/3/2018,2040.0
3,BE02-P1-LR1-FI01,Llenadora Botellas Línea 1,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIO LAMPARA UV L-1,8/3/2018,0.0
4,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIO LAMPARA UV SIST.ALIMENTACIÓNTAPAS,12/14/2018,3192.0
5,BE02-P1-LR1-FI01,Llenadora Botellas Línea 1,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIO LAMPARA UV LLEN x 1500 HORAS L-1,3/22/2019,2352.0
6,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV X1500 HORAS LLEN L2,7/16/2019,2784.0
7,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV L2,11/11/2019,2832.0
8,BE02-P1-LR1-FI01,Llenadora Botellas Línea 1,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV LLENADORA L1,2/17/2020,2352.0
9,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV HORAS DE SERVICIO,7/27/2020,3864.0


In [17]:
df['Horas de vida'].nsmallest(5)

0        0.0
3        0.0
11     264.0
1     1800.0
2     2040.0
Name: Horas de vida, dtype: float64

In [18]:
# Reordenamos en orden ascendente
datos_reordenados = df['Horas de vida'].sort_values()
# El tercer valor sera el umbral para definir las censuras
datos_reordenados.iloc[2]

264.0

In [19]:
# Censuramos los tres valores mas pequeños
umbral = datos_reordenados.iloc[2]
df['estado'] = np.where(df['Horas de vida'] <= umbral, 0, 1)

In [20]:
df

,Ubic.técn.,Descripción del Objeto Tecnico,Texto breve de material,Texto breve orden,Fecha,Horas de vida,estado
0,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV LLENADORA BOT L2,2/24/2018,0.0,0
1,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV EN LLENADORA L2,5/10/2018,1800.0,1
2,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIO LAMPARA UV L-2,8/3/2018,2040.0,1
3,BE02-P1-LR1-FI01,Llenadora Botellas Línea 1,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIO LAMPARA UV L-1,8/3/2018,0.0,0
4,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIO LAMPARA UV SIST.ALIMENTACIÓNTAPAS,12/14/2018,3192.0,1
5,BE02-P1-LR1-FI01,Llenadora Botellas Línea 1,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIO LAMPARA UV LLEN x 1500 HORAS L-1,3/22/2019,2352.0,1
6,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV X1500 HORAS LLEN L2,7/16/2019,2784.0,1
7,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV L2,11/11/2019,2832.0,1
8,BE02-P1-LR1-FI01,Llenadora Botellas Línea 1,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV LLENADORA L1,2/17/2020,2352.0,1
9,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV HORAS DE SERVICIO,7/27/2020,3864.0,1


Si eliminamos las columnas que inicialmente tenian `-` o si las censuramos y dentro de esa censura caen los valores inferiores a 2000 ya podemos inferir a simple vista que la probabilidad es 1 pues todos los registros superarian las 2000 horas. Por esto, al ser solo dos registros de 14, es de cir, $14$ % de los datos reemplacé los valores faltantes con la media, tiene sentido al no haber tenido un registro pensar que el valor medio coincide o estará cercano a la fecha en que ocurrió el evento, sin embargo, estos valores que se completaron con la media también estarán censurados dentro del algoritmo de `Kaplan Meier`

In [21]:
min_values = df['Horas de vida'].nsmallest(2)

# Reemplaza los tres valores más pequeños por None
df.loc[df['Horas de vida'].isin(min_values), 'Horas de vida'] = df['Horas de vida'].mean()

df

,Ubic.técn.,Descripción del Objeto Tecnico,Texto breve de material,Texto breve orden,Fecha,Horas de vida,estado
0,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV LLENADORA BOT L2,2/24/2018,2358.857143,0
1,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV EN LLENADORA L2,5/10/2018,1800.000000,1
2,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIO LAMPARA UV L-2,8/3/2018,2040.000000,1
3,BE02-P1-LR1-FI01,Llenadora Botellas Línea 1,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIO LAMPARA UV L-1,8/3/2018,2358.857143,0
4,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIO LAMPARA UV SIST.ALIMENTACIÓNTAPAS,12/14/2018,3192.000000,1
5,BE02-P1-LR1-FI01,Llenadora Botellas Línea 1,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIO LAMPARA UV LLEN x 1500 HORAS L-1,3/22/2019,2352.000000,1
6,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV X1500 HORAS LLEN L2,7/16/2019,2784.000000,1
7,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV L2,11/11/2019,2832.000000,1
8,BE02-P1-LR1-FI01,Llenadora Botellas Línea 1,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV LLENADORA L1,2/17/2020,2352.000000,1
9,BE02-P1-LR2-FI01,Llenadora Botellas Línea 2,LAMPARA UV -HOZFR 200 D22U 240E251,CAMBIAR LAMPARA UV HORAS DE SERVICIO,7/27/2020,3864.000000,1


In [22]:
kmf = KaplanMeierFitter()

# Ajustar el modelo Kaplan-Meier a los datos
kmf.fit(df['Horas de vida'], event_observed=df['estado'])

# Estimar la probabilidad de supervivencia
probabilidad_exito = kmf.predict(2000)

print(f'La probabilidad estimada de supervivencia en t=2000 horas: {probabilidad_exito:.4f}, es decir: {round(probabilidad_exito*100,2)} %')

La probabilidad estimada de supervivencia en t=2000 horas: 0.9231, es decir: 92.31 %


Podemos ahora calcular entonces la probabilidad de que falle $P_{exito} =1-P_{fracaso} $

In [23]:
probabilidad_fracaso = 1-probabilidad_exito
print(f'La probabilidad de que la máquina falle es: {probabilidad_fracaso:.4f}, es decir: {round(probabilidad_fracaso*100,2)} %')

La probabilidad de que la máquina falle es: 0.0769, es decir: 7.69 %
